# What is PyTorch?

このノートブックでは，近年において最も主流な深層学習フレームワークのひとつであるPyTorchについて基本的な事項を説明し，以降で様々なモデルを実装していくための基礎を獲得することを目的とします．
# 目次

0.   準備
1.   テンソル生成方法
2.   種々の演算
3.   GPUの利用方法







## 準備
**ファイルの設定**　本稿は Google Colabolatory 上で実行されることを想定して作成されています．
以降でGPUを用いた演算を解説する際の準備として，はじめにファイルの設定を変更します．

本稿を Google Colabolatory 上で開いたのち，上部メニューから「編集」→「ノートブックの設定」を選択し，ハードウェアアクセラレータをGPUに設定してください．この設定変更はそれまでに実行されていたランタイムを初期化するため，必ず最初に行うようにしてください．

以降，GPUによる演算が必要なノートブックでは適宜同様の設定を行なってください．

**PyTorchのインポート** Google Colabolatory ではPyTorchがあらかじめインストールされています．

In [1]:
import torch                   # PyTorchのインポート
print(torch.__version__)       # バージョン確認
print(torch.cuda.is_available()) # GPUが正しく利用できるかの確認
import numpy as np # numpyのインポート

1.8.1+cu101
True


## テンソルの生成方法
----
**サイズを指定したテンソル生成**

任意サイズのテンソルを生成するために，単純に領域を確保することができます．値の初期化はなされません．

以下では例としてベクトル・行列・高階テンソルの領域をそれぞれ確保しています．同様に，任意の自然数を列挙することで様々なサイズのテンソルを生成することができます．

In [2]:
# 1D vector
x = torch.empty(3)
print(x)

tensor([-1.5035e-39,  3.0679e-41,  3.3631e-44])


In [3]:
# 2D matrix
x = torch.empty(2, 3)
print(x)

tensor([[-1.5035e-39,  3.0679e-41,  5.0447e-44],
        [ 0.0000e+00,         nan,  0.0000e+00]])


In [4]:
# 3D tensor
x = torch.empty(2, 3, 2)
print(x)

tensor([[[-1.5034e-39,  3.0679e-41],
         [ 3.3631e-44,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00]]])


生成したテンソルのサイズは以下のように確認することができます．

In [5]:
print(x.size())  # テンソルのサイズを取得
print(x.size(0)) # テンソルの0次元目のサイズを取得

torch.Size([2, 3, 2])
2


また，0や1などの特定の値や任意の値で埋めたテンソルを生成することができます．

In [6]:
# 0で埋める
x = torch.zeros(2, 2)
print(x)

tensor([[0., 0.],
        [0., 0.]])


In [7]:
# 1で埋める
x = torch.ones(2, 2)
print(x)

tensor([[1., 1.],
        [1., 1.]])


In [8]:
# 任意の値（ここでは3.1415）で埋める
x = torch.full((2, 2), 3.1415)
print(x)

tensor([[3.1415, 3.1415],
        [3.1415, 3.1415]])


乱数によって初期化されたテンソルを生成することもできます．

In [9]:
# 標準正規分布で初期化
x = torch.randn(2, 2)
print(x)

tensor([[0.0498, 0.6841],
        [0.1638, 1.6404]])


In [10]:
# 0~1の一様分布で初期化
x = torch.rand(2, 2)
print(x)

tensor([[0.6241, 0.8539],
        [0.1221, 0.3954]])


----
**リストやnumpy.ndarrayからの変換**

あらかじめ定めた要素値によりテンソルを生成したい場合は，次のようにして実現できます．

In [11]:
# リストから生成
x = torch.tensor([[1., 2., 3.], [4., 5., 6.]])
print(x)

tensor([[1., 2., 3.],
        [4., 5., 6.]])


また，NumPy上でなんらかの計算を施した既存のndarrayをPyTorch上で利用したい場合も，同様のメソッドを用いることが可能です．このとき，ndarrayに設定されていた数値型がそのまま引き継がれます．

なお，print文による表示桁数は丸められてしまいますが，各要素には確かに数値型に対応する精度の数値が格納されています．

In [12]:
# 既存のndarrayとして準備
a = np.random.randn(3, 2)
print(a)
print(a.dtype)

[[-0.63152851  0.71211283]
 [ 0.259266    0.29522447]
 [ 0.19939309  0.72729284]]
float64


In [13]:
# numpy.ndarrayから生成
x = torch.tensor(a)
print(x)

tensor([[-0.6315,  0.7121],
        [ 0.2593,  0.2952],
        [ 0.1994,  0.7273]], dtype=torch.float64)


----
**要素の取り出し**
リストやNumPyのndarrayと同様，スライスにより要素を取り出すことができます．

In [14]:
# 要素の取り出し
print(x[:2, :])
print(x[0, [1, 0]])

tensor([[-0.6315,  0.7121],
        [ 0.2593,  0.2952]], dtype=torch.float64)
tensor([ 0.7121, -0.6315], dtype=torch.float64)


In [15]:
# 1要素のテンソルを単なるスカラー値に変換するために　item を用いることができる
print(x[0, 0].item())

-0.631528510369243


----
**数値型**

PyTorchにもNumPyと同様に数値型の概念が存在します．数値型はテンソル生成時に指定できるほか，一度生成したテンソルを別の型に変換することもできます．

指定しない場合は基本的にfloat型とみなされます．よく用いられる型はfloat型およびlong型です．

In [16]:
# dtypeを指定することによるfloat型での生成（numpy.float32に対応）
x = torch.ones(2, 2, dtype=torch.float)
print(x)

tensor([[1., 1.],
        [1., 1.]])


既存のリストやnumpy.ndarrayを明示的にtorch.float型のテンソルへ変換することもできます．

In [17]:
# 先ほど用意した既存のndarray
print(a)
print(a.dtype)

[[-0.63152851  0.71211283]
 [ 0.259266    0.29522447]
 [ 0.19939309  0.72729284]]
float64


In [18]:
# numpy.ndarrayからtorch.floatのテンソルを生成
x = torch.FloatTensor(a)
print(x)
print(x.dtype)

tensor([[-0.6315,  0.7121],
        [ 0.2593,  0.2952],
        [ 0.1994,  0.7273]])
torch.float32


In [19]:
# dtypeを指定することによるlong型での生成（numpy.int64に対応）
x = torch.ones(2, 2, dtype=torch.long)
print(x)

tensor([[1, 1],
        [1, 1]])


同様に，既存のリストやnumpy.ndarrayを明示的にtorch.long型のテンソルへ変換することができます．

In [20]:
# numpy.ndarrayからtorch.longのテンソルを生成
x = torch.LongTensor(a)
print(x)
print(x.dtype)

tensor([[0, 0],
        [0, 0],
        [0, 0]])
torch.int64


In [21]:
# torch.floatからtorch.longへの変換
f = torch.randn(4, 4)
print(f)

tensor([[-0.8009,  0.6617,  0.3251, -0.3302],
        [-1.4610, -0.2678,  1.1902,  0.2381],
        [-1.0265,  0.2021,  1.1383,  0.6424],
        [-0.6627,  1.4709, -2.0768,  0.6381]])


In [22]:
l = f.long()
print(l)

tensor([[ 0,  0,  0,  0],
        [-1,  0,  1,  0],
        [-1,  0,  1,  0],
        [ 0,  1, -2,  0]])


In [23]:
# torch.longからtorch.floatへの変換
f2 = l.float()
print(f2)

tensor([[ 0.,  0.,  0.,  0.],
        [-1.,  0.,  1.,  0.],
        [-1.,  0.,  1.,  0.],
        [ 0.,  1., -2.,  0.]])


---
**既存のテンソルと同サイズのテンソル生成**

様々なアルゴリズムを実装していくにあたり，同サイズのテンソルを複数用意して演算を行うという場面は頻出します．そこでPyTorchには，あるテンソルを基準に同サイズのテンソルを簡単に生成できるようにするための機能が備わっています．

これにより，全てのテンソルに同じサイズを明示的に割り当てるコードを書く冗長性の排除や，テンソルサイズを可変にしてもコードの見通しが良くなるといった利点が生まれます．

In [24]:
# 既存テンソル
x = torch.empty(2, 5, 2)
print(x.size())

torch.Size([2, 5, 2])


In [25]:
# xと同サイズの0埋めテンソルを生成
y = torch.zeros_like(x)
print(y)

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])


In [26]:
# xと同サイズの1埋めテンソルを生成
y = torch.ones_like(x)
print(y)

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])


In [27]:
# xと同サイズで任意の値で埋めたテンソルを生成
y = torch.full_like(x, 3.1415)
print(y)

tensor([[[3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415]],

        [[3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415]]])


In [28]:
# xと同サイズで標準正規分布で初期化されたテンソルを生成
y = torch.randn_like(x)
print(y)

tensor([[[ 2.0479, -0.8357],
         [-1.6240,  1.1336],
         [-0.9140,  0.2519],
         [ 1.0806,  0.5500],
         [-0.5830, -1.8972]],

        [[-1.3890, -0.7295],
         [ 0.5099,  0.1647],
         [ 0.1538, -0.9637],
         [-1.2245, -0.1549],
         [ 0.8875, -0.9610]]])


In [29]:
# xと同サイズで0~1の一様分布で初期化されたテンソルを生成
y = torch.rand_like(x)
print(y)

tensor([[[0.9901, 0.2116],
         [0.2304, 0.2951],
         [0.6731, 0.6569],
         [0.7876, 0.2013],
         [0.4038, 0.0452]],

        [[0.0302, 0.4793],
         [0.7277, 0.3743],
         [0.6767, 0.4577],
         [0.8834, 0.5060],
         [0.7030, 0.3926]]])


## 種々の演算
----
**四則演算**　基礎的な四則演算をテンソル同士で行うことができます．

In [30]:
# テンソルの準備
x = torch.randn(1, 2)
y = torch.randn(1, 2)
print(x)
print(y)

tensor([[-1.1246, -0.9232]])
tensor([[0.8873, 0.7750]])


In [31]:
# 加算
z = x + y
print(z)

tensor([[-0.2373, -0.1482]])


In [32]:
# 減算
z = x - y
print(z)

tensor([[-2.0119, -1.6982]])


In [33]:
# 要素ごとの乗算
z = x * y
print(z)

tensor([[-0.9979, -0.7155]])


In [34]:
# 要素ごとの除算
z = x / y
print(z)

tensor([[-1.2674, -1.1912]])


In [35]:
# 冪乗
z = x**2
print(z)

tensor([[1.2647, 0.8522]])


---
**利用可能な数学関数** 様々な数学関数や集計関数が利用できます

In [36]:
x = torch.randn(1,2)
print(x)

tensor([[0.1963, 0.3550]])


In [37]:
# 要素ごとの絶対値
z = torch.abs(x)
print(z)

tensor([[0.1963, 0.3550]])


In [38]:
# 要素ごとに三角関数を適用
z = torch.sin(x)
print(z)

tensor([[0.1950, 0.3476]])


In [39]:
# 要素ごとに指数関数を適用
z = torch.exp(x)
print(z)

# 要素ごとに対数関数を適用
z = torch.log(x)
print(x)

tensor([[1.2169, 1.4262]])
tensor([[0.1963, 0.3550]])


In [40]:
# 合計
z = torch.sum(x)
print(z)
# 最大値
z = torch.max(x)
print(z)
# 平均
z = torch.mean(x)
print(z)

tensor(0.5513)
tensor(0.3550)
tensor(0.2756)


----
**テンソルサイズの変更**　
PyTorchのテンソルサイズはviewを用いて任意に整形できます．

In [41]:
x = torch.randn(2, 3, 4)
print(x)
print(x.size())

tensor([[[-0.1109,  0.2830,  0.4374,  0.7467],
         [-1.4467, -1.5612,  1.1267, -0.8036],
         [ 0.2642, -0.4248,  0.7848, -0.3033]],

        [[-0.8535,  0.7119, -0.2016, -0.7923],
         [ 1.2206,  0.1530, -0.9691,  0.9518],
         [-0.3564,  0.4897,  0.6433,  0.5044]]])
torch.Size([2, 3, 4])


In [42]:
y = x.view(6, 4)
print(y)
print(y.size())

tensor([[-0.1109,  0.2830,  0.4374,  0.7467],
        [-1.4467, -1.5612,  1.1267, -0.8036],
        [ 0.2642, -0.4248,  0.7848, -0.3033],
        [-0.8535,  0.7119, -0.2016, -0.7923],
        [ 1.2206,  0.1530, -0.9691,  0.9518],
        [-0.3564,  0.4897,  0.6433,  0.5044]])
torch.Size([6, 4])


In [43]:
# -1を指定すると整合するように自動で整数を推定してくれる
y = x.view(1, 2, -1, 2, 2)
print(y)
print(y.size())

tensor([[[[[-0.1109,  0.2830],
           [ 0.4374,  0.7467]],

          [[-1.4467, -1.5612],
           [ 1.1267, -0.8036]],

          [[ 0.2642, -0.4248],
           [ 0.7848, -0.3033]]],


         [[[-0.8535,  0.7119],
           [-0.2016, -0.7923]],

          [[ 1.2206,  0.1530],
           [-0.9691,  0.9518]],

          [[-0.3564,  0.4897],
           [ 0.6433,  0.5044]]]]])
torch.Size([1, 2, 3, 2, 2])


**軸の入れ替え**　テンソルにおける任意の2軸の順番を入れ替えることができます．

In [44]:
y = x.transpose(2, 0)
print(y)
print(y.size())

tensor([[[-0.1109, -0.8535],
         [-1.4467,  1.2206],
         [ 0.2642, -0.3564]],

        [[ 0.2830,  0.7119],
         [-1.5612,  0.1530],
         [-0.4248,  0.4897]],

        [[ 0.4374, -0.2016],
         [ 1.1267, -0.9691],
         [ 0.7848,  0.6433]],

        [[ 0.7467, -0.7923],
         [-0.8036,  0.9518],
         [-0.3033,  0.5044]]])
torch.Size([4, 3, 2])


2次元行列の場合は単純に転置することも可能です．

In [45]:
m = torch.randn(3, 2)
print(m)
print(m.size())

tensor([[ 0.2644, -0.9707],
        [-1.2525, -0.5133],
        [-1.3039,  1.2381]])
torch.Size([3, 2])


In [46]:
# 転置行列を返す
n = m.t()
print(n)
print(n.size())

tensor([[ 0.2644, -1.2525, -1.3039],
        [-0.9707, -0.5133,  1.2381]])
torch.Size([2, 3])


全ての軸の順番を指定して入れ替えることも可能です．

In [47]:
# 軸の順列を指定して入れ替える
y = x.permute(2, 0, 1)
print(y)
print(y.size())

tensor([[[-0.1109, -1.4467,  0.2642],
         [-0.8535,  1.2206, -0.3564]],

        [[ 0.2830, -1.5612, -0.4248],
         [ 0.7119,  0.1530,  0.4897]],

        [[ 0.4374,  1.1267,  0.7848],
         [-0.2016, -0.9691,  0.6433]],

        [[ 0.7467, -0.8036, -0.3033],
         [-0.7923,  0.9518,  0.5044]]])
torch.Size([4, 2, 3])


**メモリの整列**　上記にあげたテンソルの整形や軸の入れ替えは任意の操作数重ね合わせることが可能です．しかしながら，軸を入れ替えると表記上のテンソルと内部で確保されているメモリの並びが不整合となる場合があり，その後にviewメソッドを用いると失敗します．

In [48]:
# view size is not compatible with input tensor's size and stride... というエラーが生じる
# 今回はこの stride の部分に問題がある
z = x.transpose(2, 0).view(2, 2, 6)

RuntimeError: ignored

このようなエラーが発生した場合は，一度メモリの並びを整列させる操作を噛ませることで解消可能です．

In [49]:
z = x.transpose(2, 0).contiguous().view(2, 2, 6)
print(z)

tensor([[[-0.1109, -0.8535, -1.4467,  1.2206,  0.2642, -0.3564],
         [ 0.2830,  0.7119, -1.5612,  0.1530, -0.4248,  0.4897]],

        [[ 0.4374, -0.2016,  1.1267, -0.9691,  0.7848,  0.6433],
         [ 0.7467, -0.7923, -0.8036,  0.9518, -0.3033,  0.5044]]])


----
**in-placeな演算**　
通常の演算では，出力テンソルとして新たなメモリ領域が確保されています．しかしながら，メモリの使用量を抑制したい場合等に，入力テンソルの要素をそのまま置換するように演算を行うことができます．

*   加算：add_
*   減算：sub_
*   乗算：mul_
*   除算：div_
*   転置：t_
*   軸の入れ替え：transpose_
*   値のコピー：copy_

等が用意されています．



In [50]:
# テンソルの準備
x = torch.randn(1, 2)
y = torch.randn(1, 2)
print(x)
print(y)

tensor([[-0.8559, -0.3378]])
tensor([[ 0.5488, -1.3168]])


In [51]:
# xにyをin-placeに加算する
x.add_(y)
# x自体の値が演算結果で書き換わる
print(x)

tensor([[-0.3071, -1.6546]])


In [52]:
# xをin-placeに転置する
x.t_()
print(x)

tensor([[-0.3071],
        [-1.6546]])


In [53]:
# xの軸をin-placeに入れ替える
x.transpose_(0,1)
print(x)

tensor([[-0.3071, -1.6546]])


In [54]:
# xにyの値をコピーする
x.copy_(y)
print(x)

tensor([[ 0.5488, -1.3168]])


ただし，後続のノートブックで説明する自動微分機能を用いる際は，基本的にin-placeな演算を行うことはできないことに留意してください．

## GPUの利用方法
----
**生成済のテンソルをGPUメモリに載せる**ことができます．複数枚のGPUが搭載されたサーバ上で実行する場合はGPU番号を指定することが推奨されますが，Google Colaboratory は1枚のみの搭載のため以下のようにして簡単に既存テンソルをGPU上のテンソル（CUDA Tensor）に変換することができます．

In [55]:
x_cuda = x.cuda()
print(x_cuda)

tensor([[ 0.5488, -1.3168]], device='cuda:0')


テンソルの生成時にdeviceを指定することで，**初めからGPU上のメモリを確保してテンソルを生成する**ことも可能です．

In [56]:
x = torch.randn(1, 2, device="cuda:0")
print(x)

tensor([[1.8219, 0.3056]], device='cuda:0')


存在しないGPU番号を指定するとエラーが発生します．

In [57]:
x = torch.randn(1, 2, device="cuda:1")

RuntimeError: ignored

同じデバイス上のテンソルを用いて，先に紹介したような演算を同様に行うことが可能です．
ただし，異なるデバイス同士（CPUとGPU，または異なる番号のGPU同士）での演算はできません．

**CPUへの変換**　GPU上のテンソルは，同様に簡単にCPU上に移行することが可能です．

In [58]:
# CPUへ
x_cpu = x_cuda.cpu()
print(x_cpu)
print(x_cpu.device)

tensor([[ 0.5488, -1.3168]])
cpu


**統一的な記法**　PyTorchが扱うテンソルは，デバイス名を文字列で指定することで簡単にデバイス間を移動させることができます．

In [59]:
# CPU上で生成
x = torch.randn(2, 2)
print(x.device)

cpu


In [60]:
# 0番GPUへ
x_cuda0 = x.to("cuda:0")
print(x_cuda0.device)

cuda:0


In [61]:
# CPUへ
x_cpu = x_cuda0.to("cpu")
print(x_cpu.device)

cpu


さらに，既存のテンソルを引数として，同じデバイス・同じ数値型への変換を行うことも可能です．

In [62]:
# 0番GPUにdouble型のテンソルを生成
d_cuda0 = torch.randn(2, 2, dtype=torch.double, device="cuda:0")
print(d_cuda0.device)
print(d_cuda0.dtype)

cuda:0
torch.float64


In [63]:
# xをd_cuda0と同じデバイス・数値型へ変換する
x_d = x.to(d_cuda0)
print(x_d.device)
print(x_d.dtype)

cuda:0
torch.float64


**他の配列オブジェクトへの変換**　CPU上のテンソルに限り，numpy.ndarrayやリストなどへ変換することができます．

In [64]:
# numpy.ndarrayに変換
x_ndarray = x_cpu.numpy()
print(x_ndarray)

[[ 0.41958195 -1.0384101 ]
 [ 1.03032    -2.5566313 ]]


In [65]:
# リストに変換
x_list = x_cpu.tolist()
print(x_list)

[[0.41958194971084595, -1.0384100675582886], [1.0303200483322144, -2.556631326675415]]


In [66]:
x = np.random.randn(3,2)
y = np.random.randn(3,2)
print(x)
print(y)
z=(x - y)**2
print(z)
s = np.sum(z, axis=1)
print(s)
m = np.mean(s)
print(m)

[[ 0.69029223  0.86272309]
 [-0.81819199 -0.36003484]
 [ 0.84917658 -0.48429939]]
[[-0.06866396 -1.50005116]
 [ 1.762917    0.50506003]
 [-0.43879353 -0.52934493]]
[[5.76014509e-01 5.58270214e+00]
 [6.66212361e+00 7.48389132e-01]
 [1.65886701e+00 2.02910125e-03]]
[6.15871665 7.41051274 1.66089611]
5.0767085012269435


# 課題
<h4>
平均二乗誤差関数の実装
</h4>

 
入力：$\boldsymbol{X}=(\boldsymbol{x}_1,\boldsymbol{x}_2,\cdots\boldsymbol{x}_{N})^{T}\in\mathbb{R}^{N\times M},　\boldsymbol{Y}=(\boldsymbol{y}_1,\boldsymbol{y}_2,\cdots\boldsymbol{y}_{N})^{T}\in\mathbb{R}^{N\times M}$
      
出力：$z\in\mathbb{R}, z=L(\boldsymbol{X}, \boldsymbol{Y}~)$
    
    

- 平均二乗誤差 :
$$ 
L(\boldsymbol{X},\boldsymbol{Y}~) = \frac{1}{N}\sum^N_i \|\boldsymbol{x}_i-\boldsymbol{y}_i\|_2^2\\
$$
Numpyのndarrayを受け取り，GPU上で平均二乗誤差を計算して，GPU上のメモリに乗ったテンソルを出力する関数を実装してみましょう


In [79]:
def MSE(x, y):
  # TODO
  x_cuda = torch.tensor(x, device="cuda:0")
  y_cuda = torch.tensor(y, device="cuda:0")
  z = ((x_cuda - y_cuda) ** 2).sum(dim=1).mean()
  
  return z

In [68]:
from google.colab import drive # driveを接続
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [69]:
# drive中の課題ファイルのあるディレクトリに移動
%cd /content/gdrive/My Drive/先端人工知能論Ⅰ/FAI20210601/data
from test import *

/content/gdrive/My Drive/先端人工知能論Ⅰ/FAI20210601/data


In [80]:
test_MSE(MSE)

OK


OKと出力されれば完成です

最終項で紹介したGPU上での演算は極めて重要であり，今日では専らGPUを用いて演算することが主流となっています．
というのも，一般の配列操作や行列演算等はGPUを用いることで高速に並列計算可能なためです．このような装置の恩恵もあって今日の深層学習モデルは現実的な時間で学習させることが可能となっています．

----
お疲れ様でした．本稿は以上で終了いたします．
以降のハンズオンでは、深層学習において重要な概念のひとつである自動微分機能について解説したのち，具体的なデータセットとタスクを通してより深層学習モデルに関する知見を深めていきます．